In [1]:
%autosave 600
%matplotlib inline
#%load_ext autoreload
#%autoreload 2
#%pdb

Autosaving every 600 seconds


In [2]:
import os
#os.environ['CUDA_VISIBLE_DEVICES'] = ""

import torch
torch.set_num_threads(4)

In [3]:
cd ..

/master


In [4]:
import os
import numpy as np
import pdb

import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, datasets

from utils.data import get_train_dev_test_ssl

from skssl.classifiers import *
from skssl.predefined import *
from skssl.training import *
from pretrained import load_pretrained

/usr/local/lib/python3.6/dist-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


# Data

In [5]:
train, dev, test = get_train_dev_test_ssl("mnist")

# Train

In [6]:
from sklearn.pipeline import Pipeline

In [7]:
vae = load_pretrained("vae", "mnist").freeze()

In [8]:
transformed_train = vae.transform(train)
transformed_dev = vae.transform(dev)
transformed_test = vae.transform(test)

In [11]:
m2 = SSLVAE(vae.module_.z_dim, train.n_classes,
            Encoder=add_flat_input(MLP, hidden_size=64, n_hidden_layers=3), 
            Decoder=MLP,
            Classifier=MLP,
            z_dim=64)

model=NeuralNetClassifier(m2, 
                   criterion=SSLVAELoss, 
                   criterion__distribution="laplace",
                   criterion__alpha=0.01,
                  max_epochs=30,
                           devset={"X":transformed_dev, "y":dev.targets})

/master/skssl/predefined/mlp.py:52: UserWarning: hidden_size=32 smaller than output=64 and input=64. Setting it to 64.
  warnings.warn(txt.format(hidden_size, output_size, input_size, self.hidden_size))
/master/skssl/predefined/mlp.py:52: UserWarning: hidden_size=64 smaller than output=128 and input=74. Setting it to 74.
  warnings.warn(txt.format(hidden_size, output_size, input_size, self.hidden_size))
/master/skssl/predefined/mlp.py:52: UserWarning: hidden_size=32 smaller than output=64 and input=74. Setting it to 64.
  warnings.warn(txt.format(hidden_size, output_size, input_size, self.hidden_size))


In [12]:
_=model.fit(transformed_train, y=train.targets)

  epoch    train_loss    valid_acc    valid_loss      dur
-------  ------------  -----------  ------------  -------
      1        0.9531       0.1123        0.0255  32.4049


      2       -2.2746       0.1132        0.0255  32.7476


      3       -2.2746       0.1123        0.0255  32.4048


      4       -2.2747       0.1138        0.0255  32.7932


      5       -2.2747       0.1138        0.0254  32.3112


# Evaluate

In [14]:
model.score(transformed_test, y=test.targets)

0.1261